In [71]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# Load data
X_train = pd.read_csv("X_train.csv", sep=';', decimal=',')
Y_train = pd.read_csv("Y_train.csv", sep=';', decimal=',')
X_test = pd.read_csv("X_test.csv", sep=';', decimal=',')
Y_test = pd.read_csv("Y_test.csv", sep=';', decimal=',')

# Merge X and Y
train = pd.merge(X_train, Y_train, on=["key", "date"])
train["date"] = pd.to_datetime(train["date"])
#train.fillna(0, inplace=True)
train['flag']='train'
X_test.fillna(0, inplace=True)
X_test['flag']='test'




In [72]:
train.shape

(8437, 116)

In [73]:
X_test["date"] = pd.to_datetime(X_test["date"])
X_test["y"]=np.nan
train=pd.concat([train,X_test])

In [74]:
# Date features
def add_date_features(df):
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["quarter"] = df["date"].dt.quarter
    return df

train = add_date_features(train)


# Sort before adding lag features
train = train.sort_values(by=["key", "date"])




In [75]:

# Add lag and rolling features
def add_lag_features(df, lags=[1, 2, 3], windows=[3, 7], ewmas=[3]):
    for lag in lags:
        df[f"lag_{lag}"] = df.groupby("key")["y"].shift(lag)
        df[f"lag_{lag}_noise"] = df[f"lag_{lag}"] + np.random.normal(0, 0.01, len(df))  # add noise

    for window in windows:
        df[f"rolling_mean_{window}"] = df.groupby("key")["y"].shift(1).rolling(window).mean().reset_index(0, drop=True)

    for span in ewmas:
        df[f"ewm_{span}"] = df.groupby("key")["y"].shift(1).ewm(span=span, adjust=False).mean().reset_index(0, drop=True)

    return df

train = add_lag_features(train)




In [76]:
#Drop highly correlated x-features
feature_cols = [col for col in train.columns if col.startswith("x")]
corr_matrix = train[feature_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
train.drop(columns=to_drop, inplace=True)

In [77]:
# Select top features correlated with y
remaining_x = [col for col in train.columns if col.startswith("x")]
cor_with_y = train[remaining_x].corrwith(train["y"]).abs()
top_10_features = cor_with_y.sort_values(ascending=False).head(10).index.tolist()

# Final features
lag_features = [col for col in train.columns if col.startswith("lag_")]
rolling_features = [col for col in train.columns if col.startswith("rolling")]
ewma_features = [col for col in train.columns if col.startswith("ewm")]

final_features = top_10_features + lag_features + rolling_features + ewma_features + ["month", "year", "quarter"]


In [78]:
final_features

['x17',
 'x101',
 'x10',
 'x88',
 'x22',
 'x109',
 'x45',
 'x53',
 'x102',
 'x5',
 'lag_1',
 'lag_1_noise',
 'lag_2',
 'lag_2_noise',
 'lag_3',
 'lag_3_noise',
 'rolling_mean_3',
 'rolling_mean_7',
 'ewm_3',
 'month',
 'year',
 'quarter']

In [79]:
# Drop NA introduced by lag features
X_test=train[train['flag']=='test']
X_test.fillna(0,inplace=True)
X_test=X_test.dropna(subset=final_features)
train=train[train['flag']=='train']
#train.fillna(0,inplace=True)
train = train.dropna(subset=final_features + ["y"])


# Time-aware split
cutoff_date = train["date"].max() - pd.DateOffset(months=3)
train_set = train[train["date"] <= cutoff_date]
val_set = train[train["date"] > cutoff_date]

# LightGBM Dataset
lgb_train = lgb.Dataset(train_set[final_features], train_set["y"])
lgb_val = lgb.Dataset(val_set[final_features], val_set["y"], reference=lgb_train)

# LightGBM Parameters
params = {
    "objective": "regression",
    "metric": "mae",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1,
    "seed": 42
}

# Train model
model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_val],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)

# Predict on validation
val_preds = model.predict(val_set[final_features])
val_mae = mean_absolute_error(val_set["y"], val_preds)
val_wmape = np.sum(np.abs(val_set["y"] - val_preds)) / np.sum(np.abs(val_set["y"])) * 100

print(f"\nValidation MAE: {val_mae:.2f}")
print(f"Validation WMAPE: {val_wmape:.2f}%")

Training until validation scores don't improve for 50 rounds
[100]	training's l1: 1021.95	valid_1's l1: 950.89
Early stopping, best iteration is:
[67]	training's l1: 1096.21	valid_1's l1: 899.872

Validation MAE: 899.87
Validation WMAPE: 15.57%


<ipython-input-79-3672386236>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0,inplace=True)


In [80]:
X_test["y"] = model.predict(X_test[final_features])


In [83]:
X_test.head()

,key,date,x1,x2,x3,x4,x5,x6,x7,x8,...,quarter,lag_1,lag_1_noise,lag_2,lag_2_noise,lag_3,lag_3_noise,rolling_mean_3,rolling_mean_7,ewm_3
0,683,2018-07-01,0.000000,0.000000,0.397238,0.000000,0.000000,0.000000,119.936184,114.063630,...,3,4670.141375,4670.128445,4716.450361,4716.438842,4257.036901,4257.037204,0.000000,0.0,0.000000
1,683,2018-08-01,0.000000,0.000000,0.000000,0.000000,74.832787,0.000000,593.803092,0.000000,...,3,0.000000,0.000000,4670.141375,4670.147749,4716.450361,4716.458770,0.000000,0.0,1430.314968
2,683,2018-09-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,222.106715,7.926074,...,3,0.000000,0.000000,0.000000,0.000000,4670.141375,4670.157178,0.000000,0.0,715.157484
3,683,2018-10-01,1.532104,0.576041,0.000000,0.000000,52.540417,21.890773,59.588630,361.870895,...,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2172.948956,0.0,2901.844692
4,683,2018-11-01,0.000000,0.000000,0.000000,2.412814,19.021238,42.032333,1970.406869,6.444104,...,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2642.095984,0.0,2869.800371


In [82]:
Y_test

,key,date,y
0,683,01Jul2018,NaN
1,683,01Aug2018,NaN
2,683,01Sep2018,NaN
3,683,01Oct2018,NaN
4,683,01Nov2018,NaN
...,...,...,...
1423,99444,01Feb2019,NaN
1424,99444,01Mar2019,NaN
1425,99444,01Apr2019,NaN
1426,99444,01May2019,NaN


In [84]:
Y_test["date1"] = pd.to_datetime(Y_test["date"])
Y_test.head()

,key,date,y,date1
0,683,01Jul2018,NaN,2018-07-01
1,683,01Aug2018,NaN,2018-08-01
2,683,01Sep2018,NaN,2018-09-01
3,683,01Oct2018,NaN,2018-10-01
4,683,01Nov2018,NaN,2018-11-01


In [86]:
df=pd.merge(X_test,Y_test,left_on=['key','date'],right_on=['key','date1'],how='right')

In [89]:
df.columns.to_list()

['key',
 'date_x',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x28',
 'x29',
 'x30',
 'x31',
 'x32',
 'x33',
 'x34',
 'x35',
 'x36',
 'x37',
 'x38',
 'x39',
 'x40',
 'x41',
 'x42',
 'x43',
 'x44',
 'x45',
 'x46',
 'x47',
 'x48',
 'x49',
 'x50',
 'x51',
 'x52',
 'x53',
 'x54',
 'x55',
 'x56',
 'x57',
 'x58',
 'x59',
 'x60',
 'x61',
 'x62',
 'x63',
 'x64',
 'x65',
 'x66',
 'x67',
 'x68',
 'x69',
 'x70',
 'x71',
 'x72',
 'x73',
 'x74',
 'x75',
 'x76',
 'x77',
 'x78',
 'x79',
 'x80',
 'x81',
 'x82',
 'x83',
 'x84',
 'x85',
 'x86',
 'x87',
 'x88',
 'x89',
 'x90',
 'x91',
 'x92',
 'x93',
 'x94',
 'x95',
 'x96',
 'x97',
 'x98',
 'x99',
 'x100',
 'x101',
 'x102',
 'x103',
 'x104',
 'x105',
 'x106',
 'x107',
 'x108',
 'x109',
 'x110',
 'x111',
 'x112',
 'y_x',
 'flag',
 'month',
 'year',
 'quarter',
 'lag_1',
 'lag_1_noise',
 'lag_2

In [92]:
data=df[['key','date_y','y_x']]
data

,key,date_y,y_x
0,683,01Jul2018,5610.925098
1,683,01Aug2018,4299.455058
2,683,01Sep2018,3146.618804
3,683,01Oct2018,3210.802314
4,683,01Nov2018,2705.240383
...,...,...,...
1423,99444,01Feb2019,3140.339598
1424,99444,01Mar2019,3140.339598
1425,99444,01Apr2019,3187.564511
1426,99444,01May2019,3187.564511


In [96]:
data.rename(columns={'date_y': 'date', 'y_x': 'y'}, inplace=True)


<ipython-input-96-1518438222>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'date_y': 'date', 'y_x': 'y'}, inplace=True)


In [97]:
data

,key,date,y
0,683,01Jul2018,5610.925098
1,683,01Aug2018,4299.455058
2,683,01Sep2018,3146.618804
3,683,01Oct2018,3210.802314
4,683,01Nov2018,2705.240383
...,...,...,...
1423,99444,01Feb2019,3140.339598
1424,99444,01Mar2019,3140.339598
1425,99444,01Apr2019,3187.564511
1426,99444,01May2019,3187.564511


In [98]:
data.to_csv("Submission.csv", sep=';', decimal=',')